In [34]:
val demo = """
            3-5
            10-14
            16-20
            12-18
            23-30

            1
            5
            8
            11
            17
            32
""".trimIndent()

In [35]:
import kotlin.io.path.Path
import kotlin.io.path.readLines
import kotlin.io.path.readText

val input = Path("puzzles/2025/day05.txt").readText()

val ranges = input.lines().takeWhile { it.isNotEmpty() }.map { it.split('-').let { (a,b) -> a.toLong() to b.toLong() } }

In [36]:
ranges.sortedBy { it.first }.fold(emptyList<Pair<Long, Long>>()) { acc, next -> 
    val prev = acc.lastOrNull() ?: return@fold listOf(next)
    when { // A: prev, B: next
        // (A----) [B~~~~]  not touching at all
        next.first > prev.second + 1 -> acc.plusElement(next)
        // (A---[B==)~~~~]  overlapping or touching
        next.second > prev.second -> acc.dropLast(1).plusElement(prev.first to next.second)
        // (A---[B==]---A)  completely overlapping
        else -> acc
    }
}

[(447008091749, 1670555330556), (2360603309607, 3723043099728), (4950367966380, 6212012481365), (6332836078245, 7699523755375), (8921484019270, 9409475595490), (12998381660604, 17957564951527), (21041355887282, 28796498371946), (32526878209367, 39003013592313), (42204294536731, 48688706545068), (53722465508822, 56711237088166), (61167866556730, 68484069473264), (73223340959484, 76299062115895), (83036356049479, 88336355492463), (93469731113983, 96965898234540), (101438079394616, 105457983779496), (105457983779498, 108365774653464), (113243675005151, 118851405579483), (121173177681726, 127288474336774), (131119709198274, 140105840661521), (143002538830426, 148177732817084), (153437460226166, 160013138307512), (162066655737986, 166911023643586), (173885028213724, 174962391238084), (174962391238086, 178650382281973), (185434338517328, 187979371917968), (195203568087762, 198855569516936), (202719323652257, 210522884417732), (211195443277855, 220831093824223), (223272245935719, 229955448275

<llm-snippet-file>day5.ipynb</llm-snippet-file>


<llm-snippet-file>day5.ipynb</llm-snippet-file>


In [37]:
import java.awt.*
import java.awt.event.ActionListener
import java.awt.event.WindowAdapter
import java.awt.event.WindowEvent
import javax.swing.*

val unsortedRanges = ranges
val sortedRanges = ranges.sortedBy { it.first }

val frame = JFrame("Range Merge Visualization")
frame.defaultCloseOperation = JFrame.DISPOSE_ON_CLOSE
frame.setSize(1000, 600)

// Global bounds for consistent scaling
val minVal = ranges.minOf { it.first }
val maxVal = ranges.maxOf { it.second }
val rangeSpan = maxVal - minVal

enum class State { SHOW_UNSORTED, SHOW_SORTED, MERGING, FINISHED }

var currentState = State.SHOW_UNSORTED
var mergeIndex = 0
var mergedList = mutableListOf<Pair<Long, Long>>()
var message = "Original Ranges (Unsorted)"

val timerDelay = 600

val panel = object : JPanel() {
    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val height = height
        val padding = 50
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan.coerceAtLeast(1)

        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString(message, 10, 25)

        val rowHeight = 25
        var y = 50

        // Draw Section: Input List (Unsorted or Sorted based on state)
        val listToDraw = if (currentState == State.SHOW_UNSORTED) unsortedRanges else sortedRanges

        listToDraw.forEachIndexed { index, range ->
            // Determine color based on state
            if (currentState == State.MERGING && index == mergeIndex) {
                g2d.color = Color.ORANGE // Currently processing
            } else if (currentState == State.MERGING && index < mergeIndex) {
                g2d.color = Color.LIGHT_GRAY // Already processed
            } else {
                g2d.color = Color.BLUE // Waiting or just displaying
            }

            val x1 = mapX(range.first)
            val x2 = mapX(range.second)
            val w = (x2 - x1).coerceAtLeast(4)

            g2d.fillRect(x1, y, w, 18)
            g2d.color = Color.BLACK
            g2d.font = Font("Monospaced", Font.PLAIN, 12)
            g2d.drawString("${range.first}-${range.second}", x1, y - 4)

            // Draw connection line to merged if processing
            if (currentState == State.MERGING && index == mergeIndex) {
                g2d.color = Color.RED
                g2d.stroke = BasicStroke(2f)
                g2d.drawRect(x1 - 2, y - 2, w + 4, 22)
            }

            y += rowHeight
        }

        // Draw Section: Merged Result
        val mergedStartY = y + 80
        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString("Merged Result:", 10, mergedStartY - 30)

        var my = mergedStartY
        mergedList.forEach { range ->
            g2d.color = Color.GREEN
            val x1 = mapX(range.first)
            val x2 = mapX(range.second)
            val w = (x2 - x1).coerceAtLeast(4)
            g2d.fillRect(x1, my, w, 18)
            g2d.color = Color.BLACK
            g2d.drawString("${range.first}-${range.second}", x1, my - 4)
            my += rowHeight
        }
    }
}

val timer = Timer(timerDelay, null)
timer.addActionListener {
    when (currentState) {
        State.SHOW_UNSORTED -> {
            // Stay here for a bit, then move to sorted
            // To make it simple, we rely on external counter or just switch after one tick logic if we wanted,
            // but let's use a simple tick counter mechanism or just state transitions.
            // Let's hold UNSORTED for 3 ticks (approx 1.8s)
            if (timer.initialDelay == 0) { // Hacky state storage or just use a counter?
                // Simplest: Just switch state.
                currentState = State.SHOW_SORTED
                message = "Sorted Ranges"
                timer.delay = 1000 // Pause before starting merge
            } else {
                timer.initialDelay = 0 // Marker that we started
            }
        }

        State.SHOW_SORTED -> {
            currentState = State.MERGING
            message = "Merging..."
            timer.delay = 600 // Normal speed
        }

        State.MERGING -> {
            if (mergeIndex < sortedRanges.size) {
                val next = sortedRanges[mergeIndex]
                val prev = mergedList.lastOrNull()

                if (prev == null) {
                    mergedList.add(next)
                } else {
                    when {
                        next.first > prev.second + 1 -> mergedList.add(next)
                        next.second > prev.second -> {
                            mergedList[mergedList.lastIndex] = prev.first to next.second
                        }

                        else -> { /* Absorbed */
                        }
                    }
                }
                mergeIndex++
            } else {
                currentState = State.FINISHED
                message = "Finished!"
                timer.stop()
            }
        }

        State.FINISHED -> {
            // No op
        }
    }
    panel.repaint()
}

// Initial delay for the first state
timer.initialDelay = 2000
timer.start()

frame.addWindowListener(object : WindowAdapter() {
    override fun windowClosed(e: WindowEvent?) {
        timer.stop()
    }
})

frame.add(panel)
frame.isVisible = true


In [39]:
// ... existing code ...
import java.awt.*
import java.awt.event.ActionListener
import java.awt.event.WindowAdapter
import java.awt.event.WindowEvent
import javax.swing.*
import kotlin.math.max

// Assume 'ranges' is already defined in previous cells
val unsortedRanges = ranges
val sortedRanges = ranges.sortedBy { it.first }

val frame = JFrame("Range Merge Visualization (Scalable)")
frame.defaultCloseOperation = JFrame.DISPOSE_ON_CLOSE
frame.setSize(1200, 800)

// Global bounds for consistent scaling
val minVal = ranges.minOf { it.first }
val maxVal = ranges.maxOf { it.second }
val rangeSpan = (maxVal - minVal).coerceAtLeast(1)

enum class State { SHOW_UNSORTED, SHOW_SORTED, MERGING, FINISHED }

var currentState = State.SHOW_UNSORTED
var mergeIndex = 0
var mergedList = mutableListOf<Pair<Long, Long>>()
var message = "Original Ranges (Unsorted)"

// Configuration for layout
val rowHeight = 20
val headerHeight = 60
val mergedHeaderHeight = 50

// Speed up if there are many items
val baseDelay = if (sortedRanges.size > 500) 5 else 50
val itemsPerTick = if (sortedRanges.size > 2000) 50 else if (sortedRanges.size > 500) 10 else 1

val panel = object : JPanel() {
    init {
        background = Color.WHITE
        isDoubleBuffered = true
    }

    // Dynamic size for ScrollPane support
    override fun getPreferredSize(): Dimension {
        val listSize = if (currentState == State.SHOW_UNSORTED) unsortedRanges.size else sortedRanges.size
        val mergedSize = mergedList.size
        // Calculate total required height
        val h = headerHeight + (listSize * rowHeight) + mergedHeaderHeight + (mergedSize * rowHeight) + 100
        return Dimension(parent?.width ?: 800, h)
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val padding = 50
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan

        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.color = Color.BLACK
        g2d.drawString(message, 10, 25)
        g2d.font = Font("Monospaced", Font.PLAIN, 12)
        g2d.drawString("Span: $minVal .. $maxVal", 10, 45)

        var y = headerHeight

        // --- Draw Input Ranges ---
        val listToDraw = if (currentState == State.SHOW_UNSORTED) unsortedRanges else sortedRanges

        // Optimization: We could calculate visible index range here, but simple painting works for <10k items
        listToDraw.forEachIndexed { index, range ->
            // Only draw if roughly visible (simple culling)
            if (y + rowHeight > visibleRect.y && y < visibleRect.y + visibleRect.height) {
                when {
                    currentState == State.MERGING && index == mergeIndex -> g2d.color = Color.ORANGE
                    currentState == State.MERGING && index < mergeIndex -> g2d.color = Color.LIGHT_GRAY
                    else -> g2d.color = Color(100, 149, 237) // Cornflower Blue
                }

                val x1 = mapX(range.first)
                val x2 = mapX(range.second)
                // Force at least 2px width so tiny ranges are visible on large scales
                val w = (x2 - x1).coerceAtLeast(2)

                g2d.fillRect(x1, y, w, rowHeight - 4)

                // Active highlight
                if (currentState == State.MERGING && index == mergeIndex) {
                    g2d.color = Color.RED
                    g2d.stroke = BasicStroke(2f)
                    g2d.drawRect(x1 - 1, y - 1, w + 2, rowHeight - 2)
                }

                // Draw text only if we have space and not too many items
                if (sortedRanges.size < 100) {
                    g2d.color = Color.BLACK
                    g2d.font = Font("Monospaced", Font.PLAIN, 10)
                    g2d.drawString("${range.first}-${range.second}", x1 + w + 5, y + rowHeight - 6)
                }
            }
            y += rowHeight
        }

        // --- Draw Merged Ranges ---
        y += mergedHeaderHeight
        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString("Merged Result (${mergedList.size}):", 10, y - 20)

        mergedList.forEach { range ->
            if (y + rowHeight > visibleRect.y && y < visibleRect.y + visibleRect.height) {
                g2d.color = Color(50, 205, 50) // Lime Green
                val x1 = mapX(range.first)
                val x2 = mapX(range.second)
                val w = (x2 - x1).coerceAtLeast(2)
                g2d.fillRect(x1, y, w, rowHeight - 4)

                if (sortedRanges.size < 100) {
                    g2d.color = Color.BLACK
                    g2d.font = Font("Monospaced", Font.PLAIN, 10)
                    g2d.drawString("${range.first}-${range.second}", x1 + w + 5, y + rowHeight - 6)
                }
            }
            y += rowHeight
        }
    }
}

val scrollPane = JScrollPane(panel)
scrollPane.verticalScrollBar.unitIncrement = 16 // Faster scrolling
frame.add(scrollPane)

val timer = Timer(baseDelay, null)
timer.addActionListener {
    when (currentState) {
        State.SHOW_UNSORTED -> {
            if (timer.initialDelay == 0) {
                currentState = State.SHOW_SORTED
                message = "Sorted Ranges"
                timer.delay = 1000
            } else {
                timer.initialDelay = 0
            }
        }
        State.SHOW_SORTED -> {
            currentState = State.MERGING
            message = "Merging..."
            timer.delay = baseDelay
        }
        State.MERGING -> {
            // Process in batches
            repeat(itemsPerTick) {
                if (mergeIndex < sortedRanges.size) {
                    val next = sortedRanges[mergeIndex]
                    val prev = mergedList.lastOrNull()

                    if (prev == null) {
                        mergedList.add(next)
                    } else {
                        when {
                            next.first > prev.second + 1 -> mergedList.add(next)
                            next.second > prev.second -> {
                                mergedList[mergedList.lastIndex] = prev.first to next.second
                            }
                            else -> { /* Absorbed */ }
                        }
                    }
                    mergeIndex++
                } else {
                    currentState = State.FINISHED
                    message = "Finished!"
                    timer.stop()
                    return@repeat
                }
            }
            // Auto-scroll to current process
            if (currentState == State.MERGING) {
                val targetY = headerHeight + (mergeIndex * rowHeight)
                val viewRect = scrollPane.viewport.viewRect
                if (targetY > viewRect.y + viewRect.height - 100) {
                    scrollPane.viewport.viewPosition = Point(0, targetY - viewRect.height + 100)
                }
            }
        }
        State.FINISHED -> {}
    }
    panel.revalidate() // Crucial for scrollpane to update size
    panel.repaint()
}

timer.initialDelay = 1500
timer.start()

frame.addWindowListener(object : WindowAdapter() {
    override fun windowClosed(e: WindowEvent?) {
        timer.stop()
    }
})

frame.isVisible = true
// ... existing code ...


In [40]:
// ... existing code ...
import java.awt.*
import java.awt.event.ActionListener
import java.awt.event.WindowAdapter
import java.awt.event.WindowEvent
import javax.swing.*
import kotlin.math.max

// Assume 'ranges' is available from previous cells
val unsortedRanges = ranges
val sortedRanges = ranges.sortedBy { it.first }

// Global bounds for consistent scaling
val minVal = ranges.minOf { it.first }
val maxVal = ranges.maxOf { it.second }
val rangeSpan = (maxVal - minVal).coerceAtLeast(1)

// Visualization State
enum class State { SHOW_UNSORTED, SHOW_SORTED, MERGING, FINISHED }
var currentState = State.SHOW_UNSORTED
var mergeIndex = 0
var mergedList = mutableListOf<Pair<Long, Long>>()
var message = "Original Ranges (Unsorted)"

// UI Layout Constants
val rowHeight = 20
val padding = 50

val frame = JFrame("Range Merge Visualization (Fixed Bottom)")
frame.defaultCloseOperation = JFrame.DISPOSE_ON_CLOSE
frame.setSize(1200, 800)
frame.layout = BorderLayout()

// --- 1. Top Panel: Scrollable Input List ---
val inputPanel = object : JPanel() {
    init {
        background = Color.WHITE
        isDoubleBuffered = true
    }

    override fun getPreferredSize(): Dimension {
        // Dynamically calculate height based on list size to enable scrolling
        val listSize = if (currentState == State.SHOW_UNSORTED) unsortedRanges.size else sortedRanges.size
        val h = 50 + (listSize * rowHeight) + 50 // header + items + buffer
        return Dimension(parent?.width ?: 800, h)
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString(message, 10, 25)

        val listToDraw = if (currentState == State.SHOW_UNSORTED) unsortedRanges else sortedRanges
        var y = 50

        // Optimization: Only loop through visible items
        // (Simple version: we iterate all, but simple culling logic helps performance)
        val clip = g.clipBounds ?: Rectangle(0, 0, width, height)

        val startIndex = ((clip.y - 50) / rowHeight).coerceAtLeast(0)
        val endIndex = (((clip.y + clip.height - 50) / rowHeight) + 1).coerceAtMost(listToDraw.size - 1)

        if(startIndex <= endIndex) {
            for (i in startIndex..endIndex) {
                val range = listToDraw[i]
                val drawY = 50 + i * rowHeight

                // Color Logic
                when {
                    currentState == State.MERGING && i == mergeIndex -> g2d.color = Color.ORANGE
                    currentState == State.MERGING && i < mergeIndex -> g2d.color = Color.LIGHT_GRAY
                    else -> g2d.color = Color(100, 149, 237) // Cornflower Blue
                }

                val x1 = mapX(range.first)
                val x2 = mapX(range.second)
                val w = (x2 - x1).coerceAtLeast(2) // Min width for visibility

                g2d.fillRect(x1, drawY, w, rowHeight - 4)

                // Highlight active being merged
                if (currentState == State.MERGING && i == mergeIndex) {
                    g2d.color = Color.RED
                    g2d.stroke = BasicStroke(2f)
                    g2d.drawRect(x1 - 1, drawY - 1, w + 2, rowHeight - 2)
                }
            }
        }
    }
}

val scrollPane = JScrollPane(inputPanel)
scrollPane.verticalScrollBar.unitIncrement = 16
frame.add(scrollPane, BorderLayout.CENTER)

// --- 2. Bottom Panel: Fixed Merged Result ---
val mergedPanel = object : JPanel() {
    init {
        preferredSize = Dimension(1200, 120)
        background = Color(245, 245, 245)
        border = BorderFactory.createCompoundBorder(
            BorderFactory.createMatteBorder(1, 0, 0, 0, Color.GRAY),
            BorderFactory.createEmptyBorder(10, 10, 10, 10)
        )
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString("Merged Ranges (Single Line View):", 10, 20)
        g2d.font = Font("Monospaced", Font.PLAIN, 12)
        g2d.drawString("Count: ${mergedList.size}", 10, 40)

        // Draw all merged ranges on a single Y line
        val barY = 60
        val barHeight = 40

        mergedList.forEach { range ->
            g2d.color = Color(50, 205, 50) // Lime Green
            val x1 = mapX(range.first)
            val x2 = mapX(range.second)
            val w = (x2 - x1).coerceAtLeast(2)

            g2d.fillRect(x1, barY, w, barHeight)

            // Optional outline
            g2d.color = Color(34, 139, 34)
            g2d.stroke = BasicStroke(1f)
            g2d.drawRect(x1, barY, w, barHeight)
        }
    }
}

frame.add(mergedPanel, BorderLayout.SOUTH)

// --- 3. Animation Logic ---

// Adaptive speed
val baseDelay = if (sortedRanges.size > 1000) 2 else if (sortedRanges.size > 200) 10 else 50
val itemsPerTick = if (sortedRanges.size > 5000) 100 else if (sortedRanges.size > 1000) 20 else 1

val timer = Timer(baseDelay, null)
timer.addActionListener {
    var repaintNeeded = false

    when (currentState) {
        State.SHOW_UNSORTED -> {
            if (timer.initialDelay == 0) {
                currentState = State.SHOW_SORTED
                message = "Sorted Ranges by Start Position"
                timer.delay = 1000
                repaintNeeded = true
            } else {
                timer.initialDelay = 0
            }
        }
        State.SHOW_SORTED -> {
            currentState = State.MERGING
            message = "Merging..."
            timer.delay = baseDelay
            repaintNeeded = true
        }
        State.MERGING -> {
            repeat(itemsPerTick) {
                if (mergeIndex < sortedRanges.size) {
                    val next = sortedRanges[mergeIndex]
                    val prev = mergedList.lastOrNull()

                    if (prev == null) {
                        mergedList.add(next)
                    } else {
                        when {
                            // Gap
                            next.first > prev.second + 1 -> mergedList.add(next)
                            // Overlap or Touch
                            next.second > prev.second -> {
                                mergedList[mergedList.lastIndex] = prev.first to next.second
                            }
                            // Absorbed (else) -> do nothing
                        }
                    }
                    mergeIndex++
                    repaintNeeded = true
                } else {
                    currentState = State.FINISHED
                    message = "Finished! Total Merged: ${mergedList.size}"
                    timer.stop()
                    repaintNeeded = true
                    return@repeat
                }
            }

            // Auto-scroll logic for the list
            if (currentState == State.MERGING) {
                val targetY = 50 + (mergeIndex * rowHeight)
                val viewRect = scrollPane.viewport.viewRect
                // Keep active item in the middle-ish if possible, or just ensure visible
                if (targetY > viewRect.y + viewRect.height - 100) {
                    val newY = (targetY - viewRect.height + 100).coerceAtLeast(0)
                    scrollPane.viewport.viewPosition = Point(0, newY)
                }
            }
        }
        State.FINISHED -> {}
    }

    if (repaintNeeded) {
        inputPanel.revalidate()
        inputPanel.repaint()
        mergedPanel.repaint()
    }
}

timer.initialDelay = 1500
timer.start()

frame.addWindowListener(object : WindowAdapter() {
    override fun windowClosed(e: WindowEvent?) {
        timer.stop()
    }
})

frame.isVisible = true
// ... existing code ...

In [42]:
// ... existing code ...
import java.awt.*
import java.awt.event.ActionListener
import java.awt.event.WindowAdapter
import java.awt.event.WindowEvent
import javax.swing.*
import kotlin.math.max

// Assume 'ranges' is available from previous cells
val unsortedRanges = ranges
val sortedRanges = ranges.sortedBy { it.first }

// Global bounds for consistent scaling
val minVal = ranges.minOf { it.first }
val maxVal = ranges.maxOf { it.second }
val rangeSpan = (maxVal - minVal).coerceAtLeast(1)

// Visualization State
enum class State { SHOW_UNSORTED, SHOW_SORTED, MERGING, FINISHED }
var currentState = State.SHOW_UNSORTED
var mergeIndex = 0
var mergedList = mutableListOf<Pair<Long, Long>>()
var message = "Original Ranges (Unsorted)"

// UI Layout Constants
val rowHeight = 20
val padding = 50

val frame = JFrame("Range Merge Visualization (Fixed Bottom)")
frame.defaultCloseOperation = JFrame.DISPOSE_ON_CLOSE
frame.setSize(1200, 800)
frame.layout = BorderLayout()

// --- 1. Top Panel: Scrollable Input List ---
val inputPanel = object : JPanel() {
    init {
        background = Color.WHITE
        isDoubleBuffered = true
    }

    override fun getPreferredSize(): Dimension {
        // Dynamically calculate height based on list size to enable scrolling
        val listSize = if (currentState == State.SHOW_UNSORTED) unsortedRanges.size else sortedRanges.size
        val h = 50 + (listSize * rowHeight) + 50 // header + items + buffer
        return Dimension(parent?.width ?: 800, h)
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString(message, 10, 25)

        val listToDraw = if (currentState == State.SHOW_UNSORTED) unsortedRanges else sortedRanges

        // Optimization: Draw only visible items (basic culling)
        val clip = g.clipBounds ?: Rectangle(0, 0, width, height)
        val startIndex = ((clip.y - 50) / rowHeight).coerceAtLeast(0)
        val endIndex = (((clip.y + clip.height - 50) / rowHeight) + 1).coerceAtMost(listToDraw.size - 1)

        if(startIndex <= endIndex) {
            for (i in startIndex..endIndex) {
                val range = listToDraw[i]
                val drawY = 50 + i * rowHeight

                // Color Logic
                when {
                    currentState == State.MERGING && i == mergeIndex -> g2d.color = Color.ORANGE
                    currentState == State.MERGING && i < mergeIndex -> g2d.color = Color.LIGHT_GRAY
                    else -> g2d.color = Color(100, 149, 237) // Cornflower Blue
                }

                val x1 = mapX(range.first)
                val x2 = mapX(range.second)
                val w = (x2 - x1).coerceAtLeast(2)

                g2d.fillRect(x1, drawY, w, rowHeight - 4)

                // Highlight active being merged
                if (currentState == State.MERGING && i == mergeIndex) {
                    g2d.color = Color.RED
                    g2d.stroke = BasicStroke(2f)
                    g2d.drawRect(x1 - 1, drawY - 1, w + 2, rowHeight - 2)
                }
            }
        }
    }
}

val scrollPane = JScrollPane(inputPanel)
scrollPane.verticalScrollBar.unitIncrement = 16
frame.add(scrollPane, BorderLayout.CENTER)

// --- 2. Bottom Panel: Fixed Merged Result ---
val mergedPanel = object : JPanel() {
    init {
        preferredSize = Dimension(1200, 120)
        background = Color(245, 245, 245)
        border = BorderFactory.createCompoundBorder(
            BorderFactory.createMatteBorder(1, 0, 0, 0, Color.GRAY),
            BorderFactory.createEmptyBorder(10, 10, 10, 10)
        )
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString("Merged Ranges (Single Line View):", 10, 20)

        // Calculate total size (sum of all merged ranges)
        val totalSize = mergedList.sumOf { it.second - it.first + 1 }

        g2d.font = Font("Monospaced", Font.PLAIN, 12)
        g2d.drawString("Count: ${mergedList.size}   Total Covered: $totalSize", 10, 40)

        // Draw all merged ranges on a single Y line
        val barY = 60
        val barHeight = 40

        mergedList.forEach { range ->
            g2d.color = Color(50, 205, 50) // Lime Green
            val x1 = mapX(range.first)
            val x2 = mapX(range.second)
            val w = (x2 - x1).coerceAtLeast(2)

            g2d.fillRect(x1, barY, w, barHeight)

            // Optional outline
            g2d.color = Color(34, 139, 34)
            g2d.stroke = BasicStroke(1f)
            g2d.drawRect(x1, barY, w, barHeight)
        }
    }
}

frame.add(mergedPanel, BorderLayout.SOUTH)

// --- 3. Animation Logic ---

// Adaptive speed
val baseDelay = if (sortedRanges.size > 1000) 2 else if (sortedRanges.size > 200) 10 else 50
val itemsPerTick = if (sortedRanges.size > 5000) 100 else if (sortedRanges.size > 1000) 20 else 1

val timer = Timer(baseDelay, null)
timer.addActionListener {
    var repaintNeeded = false

    when (currentState) {
        State.SHOW_UNSORTED -> {
            if (timer.initialDelay == 0) {
                currentState = State.SHOW_SORTED
                message = "Sorted Ranges by Start Position"
                timer.delay = 1000
                repaintNeeded = true
            } else {
                timer.initialDelay = 0
            }
        }
        State.SHOW_SORTED -> {
            currentState = State.MERGING
            message = "Merging..."
            timer.delay = baseDelay
            repaintNeeded = true
        }
        State.MERGING -> {
            repeat(itemsPerTick) {
                if (mergeIndex < sortedRanges.size) {
                    val next = sortedRanges[mergeIndex]
                    val prev = mergedList.lastOrNull()

                    if (prev == null) {
                        mergedList.add(next)
                    } else {
                        when {
                            // Gap
                            next.first > prev.second + 1 -> mergedList.add(next)
                            // Overlap or Touch
                            next.second > prev.second -> {
                                mergedList[mergedList.lastIndex] = prev.first to next.second
                            }
                            // Absorbed (else) -> do nothing
                        }
                    }
                    mergeIndex++
                    repaintNeeded = true
                } else {
                    currentState = State.FINISHED
                    message = "Finished! Total Merged: ${mergedList.size}"
                    timer.stop()
                    repaintNeeded = true
                    return@repeat
                }
            }

            // Auto-scroll logic for the list
            if (currentState == State.MERGING) {
                val targetY = 50 + (mergeIndex * rowHeight)
                val viewRect = scrollPane.viewport.viewRect
                if (targetY > viewRect.y + viewRect.height - 100) {
                    val newY = (targetY - viewRect.height + 100).coerceAtLeast(0)
                    scrollPane.viewport.viewPosition = Point(0, newY)
                }
            }
        }
        State.FINISHED -> {}
    }

    if (repaintNeeded) {
        inputPanel.revalidate()
        inputPanel.repaint()
        mergedPanel.repaint()
    }
}

timer.initialDelay = 1500
timer.start()

frame.addWindowListener(object : WindowAdapter() {
    override fun windowClosed(e: WindowEvent?) {
        timer.stop()
    }
})

frame.isVisible = true
// ... existing code ...

In [43]:
// ... existing code ...
import java.awt.*
import java.awt.event.ActionListener
import java.awt.event.WindowAdapter
import java.awt.event.WindowEvent
import javax.swing.*
import kotlin.math.max

// Assume 'ranges' is available from previous cells
val unsortedRanges = ranges
// We still keep reference sorted list for speed validation if needed, but we'll sort visually
val finalSortedRanges = ranges.sortedBy { it.first }

// Backing list for the UI - starts unsorted, gets modified in place
val displayList = unsortedRanges.toMutableList()

// Global bounds for consistent scaling
val minVal = ranges.minOf { it.first }
val maxVal = ranges.maxOf { it.second }
val rangeSpan = (maxVal - minVal).coerceAtLeast(1)

// Visualization State
enum class State { SHOW_UNSORTED, SORTING, MERGED_WAIT, MERGING, FINISHED }
var currentState = State.SHOW_UNSORTED
var message = "Original Ranges (Unsorted)"

// Sorting State Variables
var sortI = 0 // The boundary of the sorted section (top)
var sortJ = displayList.size - 1 // The scanning index (moves bottom -> top)

// Merging State Variables
var mergeIndex = 0
var mergedList = mutableListOf<Pair<Long, Long>>()

// UI Layout Constants
val rowHeight = 20
val padding = 50

val frame = JFrame("Range Merge Visualization (Animated Sort)")
frame.defaultCloseOperation = JFrame.DISPOSE_ON_CLOSE
frame.setSize(1200, 800)
frame.layout = BorderLayout()

// --- 1. Top Panel: Scrollable List (Unsorted -> Sorted) ---
val inputPanel = object : JPanel() {
    init {
        background = Color.WHITE
        isDoubleBuffered = true
    }

    override fun getPreferredSize(): Dimension {
        val listSize = displayList.size
        val h = 50 + (listSize * rowHeight) + 50
        return Dimension(parent?.width ?: 800, h)
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString(message, 10, 25)

        // Optimization: Culling
        val clip = g.clipBounds ?: Rectangle(0, 0, width, height)
        val startIndex = ((clip.y - 50) / rowHeight).coerceAtLeast(0)
        val endIndex = (((clip.y + clip.height - 50) / rowHeight) + 1).coerceAtMost(displayList.size - 1)

        if(startIndex <= endIndex) {
            for (i in startIndex..endIndex) {
                val range = displayList[i]
                val drawY = 50 + i * rowHeight

                // Color Logic
                when (currentState) {
                    State.SORTING -> {
                        if (i < sortI) g2d.color = Color(100, 149, 237) // Sorted (Blue)
                        else if (i == sortJ || i == sortJ - 1) g2d.color = Color.ORANGE // Active Compare
                        else g2d.color = Color.LIGHT_GRAY // Unsorted
                    }
                    State.MERGING -> {
                        if (i == mergeIndex) g2d.color = Color.ORANGE // Active Merge
                        else if (i < mergeIndex) g2d.color = Color.LIGHT_GRAY // Processed
                        else g2d.color = Color(100, 149, 237) // Pending
                    }
                    else -> g2d.color = Color(100, 149, 237)
                }

                val x1 = mapX(range.first)
                val x2 = mapX(range.second)
                val w = (x2 - x1).coerceAtLeast(2)

                g2d.fillRect(x1, drawY, w, rowHeight - 4)

                // Highlight border for active items
                if ((currentState == State.MERGING && i == mergeIndex) ||
                    (currentState == State.SORTING && (i == sortJ || i == sortJ - 1))) {
                    g2d.color = Color.RED
                    g2d.stroke = BasicStroke(2f)
                    g2d.drawRect(x1 - 1, drawY - 1, w + 2, rowHeight - 2)
                }
            }
        }
    }
}

val scrollPane = JScrollPane(inputPanel)
scrollPane.verticalScrollBar.unitIncrement = 16
frame.add(scrollPane, BorderLayout.CENTER)

// --- 2. Bottom Panel: Fixed Merged Result ---
val mergedPanel = object : JPanel() {
    init {
        preferredSize = Dimension(1200, 120)
        background = Color(245, 245, 245)
        border = BorderFactory.createCompoundBorder(
            BorderFactory.createMatteBorder(1, 0, 0, 0, Color.GRAY),
            BorderFactory.createEmptyBorder(10, 10, 10, 10)
        )
    }

    override fun paintComponent(g: Graphics) {
        super.paintComponent(g)
        val g2d = g as Graphics2D
        g2d.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON)

        val width = width
        val scaleX = (width - 2 * padding).toDouble() / rangeSpan
        fun mapX(value: Long): Int = (padding + (value - minVal) * scaleX).toInt()

        g2d.color = Color.BLACK
        g2d.font = Font("SansSerif", Font.BOLD, 16)
        g2d.drawString("Merged Ranges (Single Line View):", 10, 20)

        val totalSize = mergedList.sumOf { it.second - it.first + 1 }
        g2d.font = Font("Monospaced", Font.PLAIN, 12)
        g2d.drawString("Count: ${mergedList.size}   Total Covered: $totalSize", 10, 40)

        val barY = 60
        val barHeight = 40

        mergedList.forEach { range ->
            g2d.color = Color(50, 205, 50) // Lime Green
            val x1 = mapX(range.first)
            val x2 = mapX(range.second)
            val w = (x2 - x1).coerceAtLeast(2)
            g2d.fillRect(x1, barY, w, barHeight)
            g2d.color = Color(34, 139, 34)
            g2d.stroke = BasicStroke(1f)
            g2d.drawRect(x1, barY, w, barHeight)
        }
    }
}

frame.add(mergedPanel, BorderLayout.SOUTH)

// --- 3. Animation Logic ---
// Dynamic speed calculation
val n = displayList.size
// Sorting complexity is N^2. We want to finish in ~3 seconds (approx 180 ticks at 16ms).
// Ops per tick = (N*N/2) / 180
val sortOpsTotal = n.toLong() * n / 2
val sortOpsPerTick = (sortOpsTotal / 180).toInt().coerceIn(1, 50000)

// Merging complexity is N. Finish in ~3 seconds.
val mergeOpsPerTick = (n / 180).coerceIn(1, 100)

val timer = Timer(16, null) // ~60 FPS target
timer.addActionListener {
    var repaintNeeded = false

    when (currentState) {
        State.SHOW_UNSORTED -> {
            if (timer.initialDelay == 0) {
                currentState = State.SORTING
                message = "Sorting Ranges (Sinking Sort)..."
                // Initialize sort variables
                sortI = 0
                sortJ = displayList.size - 1
                repaintNeeded = true
            } else {
                timer.initialDelay = 0
            }
        }

        State.SORTING -> {
            var ops = 0
            while (ops < sortOpsPerTick && currentState == State.SORTING) {
                if (sortI < displayList.size - 1) {
                    if (sortJ > sortI) {
                        // Compare adjacent
                        if (displayList[sortJ].first < displayList[sortJ-1].first) {
                            val tmp = displayList[sortJ]
                            displayList[sortJ] = displayList[sortJ-1]
                            displayList[sortJ-1] = tmp
                        }
                        sortJ--
                    } else {
                        // Finished one pass, item at sortI is settled
                        sortI++
                        sortJ = displayList.size - 1
                    }
                    ops++
                } else {
                    currentState = State.MERGED_WAIT
                    message = "Sorted! Ready to merge."
                    timer.delay = 1000
                    repaintNeeded = true
                }
            }
            // Scroll to follow the sorted boundary 'sortI'
            if (currentState == State.SORTING) {
                val targetY = 50 + (sortI * rowHeight)
                val viewRect = scrollPane.viewport.viewRect
                if (targetY > viewRect.y + viewRect.height / 2) {
                    val newY = (targetY - viewRect.height / 2).coerceAtLeast(0)
                    scrollPane.viewport.viewPosition = Point(0, newY)
                }
            }
            repaintNeeded = true
        }

        State.MERGED_WAIT -> {
            currentState = State.MERGING
            message = "Merging..."
            timer.delay = 16 // Back to fast speed
            // Reset scroll to top
            scrollPane.viewport.viewPosition = Point(0, 0)
        }

        State.MERGING -> {
            repeat(mergeOpsPerTick) {
                if (mergeIndex < displayList.size) {
                    val next = displayList[mergeIndex]
                    val prev = mergedList.lastOrNull()

                    if (prev == null) {
                        mergedList.add(next)
                    } else {
                        when {
                            next.first > prev.second + 1 -> mergedList.add(next)
                            next.second > prev.second -> {
                                mergedList[mergedList.lastIndex] = prev.first to next.second
                            }
                        }
                    }
                    mergeIndex++
                    repaintNeeded = true
                } else {
                    currentState = State.FINISHED
                    message = "Finished! Final Count: ${mergedList.size}"
                    timer.stop()
                    repaintNeeded = true
                    return@repeat
                }
            }

            if (currentState == State.MERGING) {
                val targetY = 50 + (mergeIndex * rowHeight)
                val viewRect = scrollPane.viewport.viewRect
                // Keep active item visible
                if (targetY > viewRect.y + viewRect.height - 100) {
                    val newY = (targetY - viewRect.height + 100).coerceAtLeast(0)
                    scrollPane.viewport.viewPosition = Point(0, newY)
                }
            }
        }
        State.FINISHED -> {}
    }

    if (repaintNeeded) {
        inputPanel.revalidate()
        inputPanel.repaint()
        mergedPanel.repaint()
    }
}

timer.initialDelay = 1500
timer.start()

frame.addWindowListener(object : WindowAdapter() {
    override fun windowClosed(e: WindowEvent?) {
        timer.stop()
    }
})

frame.isVisible = true
// ... existing code ...